In [1]:
import random

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

from pprint import pprint

import numpy

In [2]:
TAM_CROMOSSOMO = 64
TAM_POPULACAO = 1000

def fitness(cromossomo):
    """Funcao fitness para maximizar o cromossomo
    Deve retornar uma tupla!!!
    """
    count=0
    fit=0
    for i in range(TAM_CROMOSSOMO):
        if cromossomo[i] == 1:
            count += 1
            
    # Máximo 8 rainhas por cromossomo
    if (count != 8):
        return 99999,
    
    for i in(range(TAM_CROMOSSOMO)):
        if cromossomo[i] == 1:
            # Avalia as diagonais, acima e abaixo para ver se encontra alguma rainha
            fit += avalia(cromossomo, i, -9)
            fit += avalia(cromossomo, i, -8)
            fit += avalia(cromossomo, i, -7)
            fit += avalia(cromossomo, i, 7)
            fit += avalia(cromossomo, i, 8)
            fit += avalia(cromossomo, i, 9)
            
            # Avalia as lateriais
            for j in range(7):
                prox = i * j
                ant  = i * j * -1
                if (prox < TAM_CROMOSSOMO):
                    fit += cromossomo[prox]
                if (ant >= 0):
                    fit += cromossomo[ant]
    return fit,

def populacao_inicial(tam_populacao, tam_cromossomo):
    populacao = []

    for _ in range(tam_populacao):
        item = [0 for _ in range(tam_cromossomo)]
        for i in range(8):
                item[random.randint(0, 63)] = 1
                populacao.append(item)
    
    return populacao

def avalia(cromossomo, pos_atual, fat_ava):
    # Está no canto esquerdo, então não tem diagonal
    if (fat_ava == -9 or fat_ava == 7) and i%8 == 0:
        return 0
    
    # Está no canto direito, então não tem diagonal
    if (fat_ava == -7 or fat_ava == -9) and i%7 == 0:
        return 0
    
    # Indica que está na ultima linha e não tem próxima
    if fat_ava > 0 and (i + fat_ava > TAM_CROMOSSOMO):
        return 0
    
    # Indica que está na primeira linha
    if fat_ava < 0 and (i + fat_ava < 0):
        return 0
    
    iteracoes = (TAM_CROMOSSOMO - i) / abs(fat_ava)
    soma = 0
    for i in iteracoes:
        if cromossomo[i * fat_ava] == 1:
            soma += 1
    return soma

In [3]:
def varAnd(population, toolbox, cxpb, mutpb):
    offspring = [toolbox.clone(ind) for ind in population]

    # Apply crossover and mutation on the offspring
    for i in range(1, len(offspring), 2):
        if random.random() < cxpb:
            offspring[i - 1], offspring[i] = toolbox.mate(offspring[i - 1],
                                                          offspring[i])
            del offspring[i - 1].fitness.values, offspring[i].fitness.values

    for i in range(len(offspring)):
        if random.random() < mutpb:
            offspring[i], = toolbox.mutate(offspring[i])
            del offspring[i].fitness.values

    return offspring

def eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is not None:
        halloffame.update(population)

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print (logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):
        # Gera os filhos
        offspring = toolbox.select(population, len(population))

        # Mutação
        offspring = varAnd(offspring, toolbox, cxpb, mutpb)

        # Avalia os fitness inválidos
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Update the hall of fame with the generated individuals
        if halloffame is not None:
            halloffame.update(offspring)

        # Seleciona usando o algoritmo usado nos filhos
        total = population + offspring
        population = toolbox.select(total, int(len(total)/2))

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print (logbook.stream)

    return population, logbook

In [4]:
# Define a estrategia do fitness
# - weights: define se o problema é de maximizacao (+1) ou minimizacao (-1)
creator.create("FitnessMax", base.Fitness, weights=(-1.0,))

# Define a estrutura do cromossomo
creator.create("Individual", list, fitness=creator.FitnessMax)

In [5]:
# Define os componentes para configurar a populacao
toolbox = base.Toolbox()

# Gerador para os individuos
toolbox.register("attr_bool", random.randint, 0, 1)

# Inicializador da populacao
toolbox.register("individual", 
                 tools.initRepeat, 
                 creator.Individual, 
                 toolbox.attr_bool, TAM_CROMOSSOMO)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [6]:
# Define os operadores geneticos
toolbox.register("evaluate", fitness)

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("select", tools.selRoulette)

In [7]:
# Cria a populacao inicial
populacao = populacao_inicial(TAM_POPULACAO, TAM_CROMOSSOMO)

In [8]:
hof = tools.HallOfFame(10)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("std", numpy.std)
stats.register("min", numpy.min)
stats.register("max", numpy.max)
    
#pop, log = algorithms.eaSimple(populacao, 
pop, log = eaSimple(populacao, 
                               toolbox, 
                               cxpb=1, 
                               mutpb=0.2, 
                               ngen=100, 
                               stats=stats, 
                               halloffame=hof, 
                               verbose=True)

melhor = sorted([(x, x.fitness.values) for x in pop], key=lambda x: x[1], reverse= True)
print(melhor[0])

AttributeError: 'list' object has no attribute 'fitness'